<a href="https://colab.research.google.com/github/IvanStukalov/MLTechnologies/blob/lab_6/lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа 6. Ансамбли моделей машинного обучения. Часть 2.

In [2]:
!pip install heamy

  Preparing metadata (setup.py) ... done
  Created wheel for heamy: filename=heamy-0.0.7-py2.py3-none-any.whl size=15347 sha256=4cf9789f88bc55e94f307e714a9c249944d721d065c8565066edf2dd59b828eb
  Stored in directory: /root/.cache/pip/wheels/e5/e4/9a/bc85119b96421369998ff0f53c0854b57bfb518c460fe8c5de
Successfully built heamy


In [3]:
from sklearn.datasets import load_wine, load_iris
from sklearn.model_selection import train_test_split
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from heamy.dataset import Dataset
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score


### Выберите набор данных (датасет) для решения задачи классификации или регресии.

In [4]:
# Используем датасет wine
wine = load_wine()
wine_X = wine.data
wine_y = wine.target

### С использованием метода train_test_split разделите выборку на обучающую и тестовую.

In [5]:
wine_X_train, wine_X_test, wine_y_train, wine_y_test = train_test_split(
    wine_X, wine_y, test_size=0.2, random_state=1)

### Стекинг

In [6]:
# Используем библиотеку heamy
# набор данных
dataset = Dataset(wine_X_train, wine_y_train, wine_X_test)

# модели первого уровня
model_tree = Regressor(dataset=dataset, estimator=DecisionTreeRegressor, name='tree')
model_lr = Regressor(dataset=dataset, estimator=LinearRegression, name='lr')
model_rf = Regressor(dataset=dataset, estimator=RandomForestRegressor, parameters={'n_estimators': 50}, name='rf')

In [7]:
# Эксперимент 1
# Первый уровень - две модели: дерево и линейная регрессия
# Второй уровень: линейная регрессия

pipeline = ModelsPipeline(model_tree, model_lr)
stack_ds = pipeline.stack(k=10, seed=1)
# модель второго уровня
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
results = stacker.validate(k=10,scorer=mean_absolute_error)

Metric: mean_absolute_error
Folds accuracy: [0.14366357347211464, 0.20629198841959795, 0.1844558548934073, 0.1365196672185216, 0.16278448974557463, 0.18044284300760358, 0.3659790314893313, 0.11904927550480686, 0.13128012492285818, 0.22722949357237662]
Mean accuracy: 0.18576963422461928
Standard Deviation: 0.06840549970778417
Variance: 0.004679312390271661


### модель многослойного персептрона

In [8]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Создание датасета
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)

# Разделение данных на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение параметров MLP
mlp_params = {
    'hidden_layer_sizes': (50, 50), # Количество нейронов в скрытых слоях
    'activation': 'relu', # Функция активации
    'solver': 'adam', # Оптимизатор
    'alpha': 0.0001, # Коэффициент регуляризации L2
    'learning_rate': 'constant', # Стратегия изменения скорости обучения
    'learning_rate_init': 0.001, # Начальная скорость обучения
    'max_iter': 200, # Максимальное количество итераций
    'random_state': 42, # Случайное состояние для воспроизводимости
}

# Создание и обучение MLP
mlp = MLPClassifier(**mlp_params)
mlp.fit(wine_X_train, wine_y_train)

# Предсказание на тестовом наборе данных
y_pred = mlp.predict(wine_X_test)

# Оценка качества модели
accuracy = accuracy_score(wine_y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8611111111111112


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### МГУА

In [21]:
!pip install git+https://github.com/bauman-team/GMDH.git
!pip install gmdh
import gmdh

  Cloning https://github.com/bauman-team/GMDH.git to /tmp/pip-req-build-gqfbb5q7
  Running command git clone --filter=blob:none --quiet https://github.com/bauman-team/GMDH.git /tmp/pip-req-build-gqfbb5q7
  Resolved https://github.com/bauman-team/GMDH.git to commit dddc7b9a76b0930d08ac44bd3d7444473552ddb5
  Preparing metadata (setup.py) ... done
  Using cached docstring_inheritance-2.2.0-py3-none-any.whl (24 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gmdh
  Running setup.py clean for gmdh
Failed to build gmdh
ERROR: Could not build wheels for gmdh, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.3/875.3 kB 4.9 MB/s eta 0:00:00
  Using cached docstring_inheritance-2.2.0-py3-none-any.whl (24 kB)


In [26]:
def print_metrics(y_test, y_pred, squared=False):
    print(f"R^2: {r2_score(y_test, y_pred)}")
    crit_name = "MSE" if squared else "RMSE"
    print(f"{crit_name}: {mean_squared_error(y_test, y_pred, squared=squared)}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
combi_model = gmdh.Combi()
combi_model.fit(wine_X_train, wine_y_train, verbose=1, n_jobs=-1, test_size=0.24, limit=0,
                       criterion=gmdh.Criterion(gmdh.CriterionType.REGULARITY))
print()
print(combi_model.get_best_polynomial())
print()
y_pred_combi = combi_model.predict(wine_X_test)

print_metrics(wine_y_test, y_pred_combi)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LEVEL 1  [=========================] 100% :00s] (13 combinations) error=4.694871
LEVEL 2  [=========================] 100% :00s] (78 combinations) error=3.713584
LEVEL 3  [=========================] 100% :00s] (286 combinations) error=2.77371
LEVEL 4  [=========================] 100% :00s] (715 combinations) error=2.325969
LEVEL 5  [=========================] 100% :00s] (1287 combinations) error=2.199924
LEVEL 6  [=========================] 100% :00s] (1716 combinations) error=2.169235
LEVEL 7  [=========================] 100% :00s] (1716 combinations) error=2.168724
LEVEL 8  [=========================] 100% :00s] (1287 combinations) error=2.188381

y = 0.0337*x4 + 0.0187*x6 - 0.3757*x7 - 0.0294*x8 + 0.088*x10 - 0.2374*x11 - 0.001*x13 + 1.554

R^2: 0.861338679731356
RMSE: 0.2931115535963
MAE: 0.2270581109591602


In [34]:
ria_model = gmdh.Ria()
ria_model.fit(wine_X_train, wine_y_train, verbose=1, n_jobs=-1, test_size=0.52, limit=0, k_best=7,
                     criterion=gmdh.Criterion(gmdh.CriterionType.REGULARITY),
                     polynomial_type=gmdh.PolynomialType.LINEAR)
y_pred_ria = ria_model.predict(wine_X_test)
print_metrics(wine_y_test, y_pred_ria)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LEVEL 1  [=========================] 100% :00s] (78 combinations) error=11.237277
LEVEL 2  [=========================] 100% :00s] (91 combinations) error=9.196032
LEVEL 3  [=========================] 100% :00s] (91 combinations) error=7.506619
LEVEL 4  [=========================] 100% :00s] (91 combinations) error=6.087672
LEVEL 5  [=========================] 100% :00s] (91 combinations) error=5.803263
LEVEL 6  [=========================] 100% :00s] (91 combinations) error=5.480924
LEVEL 7  [=========================] 100% :00s] (91 combinations) error=5.055742
LEVEL 8  [=========================] 100% :00s] (91 combinations) error=4.804413
LEVEL 9  [=========================] 100% :00s] (91 combinations) error=4.616872
LEVEL 10 [=========================] 100% :00s] (91 combinations) error=4.525378
LEVEL 11 [=========================] 100% :00s] (91 combinations) error=4.506103
LEVEL 12 [=========================] 100% :00s] (91 combinations) error=4.498368
LEVEL 13 [=================